# Blank Factory

## Subroutines

In [1]:
import numpy as np
from matplotlib import pyplot as plt
from cmath import pi,acos,exp,sqrt
def plotArc(ax,P0,n0,l,da,tol=0.001,c=None):
  x=np.linspace(0,l,max(2,int(abs(6*(da/(2*pi)))),int(l//(2*abs(2*l/da*tol)**0.5)+1))if (da!=0) and (l!=0) else 2)
  phi2=x/l*da/2
  p=P0+x*np.sinc(phi2/pi)*n0*np.exp(1j*phi2)
  if c==None:
    ax.plot(p.real,p.imag)
  else:
    ax.plot(p.real,p.imag,c)
    
def plotArcchain(ax,P0,n0,arcs,c=None):
    p=P0
    n=n0
    for l,da in arcs:
        plotArc(ax,p,n,l,da,c=c)
        p+=l*np.sinc(da/(2*pi))*n*exp(1j*da/2)
        n*=exp(1j*da)

In [2]:
import matplotlib.pyplot as plt
import numpy
import cmath  

%matplotlib notebook

Data=dict(l1=0.15, r1=0.1, l2=0.15, r2=0.1,strands=5)
def update_plot():
  l1,r1,l2,r2,strands= [Data[key] for key in ('l1','r1','l2','r2','strands')]
  output1.clear_output(wait=True)
  with output1:
    %matplotlib inline
    fig=plt.figure(figsize=(3,3)) 
    ax=fig.add_subplot(1,1,1)  
    plotArcchain(ax,0+0j,1+0j,((l1,0),(r1*2*pi,2*pi)))
    c1=l1+1j*r1
    ax.plot(c1.real,c1.imag,('k+'))
    plotArcchain(ax,1+1j,-1+0j,((l2,0),(r2*2*pi,2*pi)))
    c2=(1-l2)+1j*(1-r2)
    ax.plot(c2.real,c2.imag,'k+') 
    c2_c1=(c2-c1)
    lcc=abs(c2_c1) 
    ec2_c1=c2_c1/lcc
    cosphi=((r1+r2)/lcc)
    sinphi=sqrt(1-cosphi**2)
    phi=cosphi+1j*sinphi
    t1=c1+r1*ec2_c1/phi
    t2=c2-r2*ec2_c1/phi
    ax.plot([t1.real,t2.real],[t1.imag,t2.imag])  
    ax.set_aspect('equal')
    ax.set_aspect(1.0)
    display(fig)
    plt.close()
  output2.clear_output(wait=True)
  with output2:
    fig=plt.figure(figsize=(12,3 )) 
    ax=fig.add_subplot(1,1,1)  
    ltan,phitan=cmath.polar(t2-t1)
    arcs=[(l1,0),(r1*phitan,phitan),(ltan,0),(r2*phitan,-phitan),(l2,0)]
    arcs=(arcs+arcs[-1::-1])*2
    for i in range(strands):
      p0=2*i/strands+0j
      n0=1+0j
      plotArcchain(ax,p0,n0,arcs,c=['r','g','b','y','k'][i])
    ax.set_xlim(0,4)
    ax.set_aspect('equal')
    display(fig)
    plt.close()
  output3.clear_output(wait=True)
  with output3:
    print(f'{l1=:0.3f}, {r1=:0.3f}, {l2=:0.3f}, {r2=:0.3f}, {strands=:d}')
  plt.close()

from ipywidgets import widgets,HBox,VBox
output1=widgets.Output() 
output2=widgets.Output()
output3=widgets.Output()
def handle_change(msg):
  Data[msg['owner'].description] = msg['new']
  update_plot()

widgetList=dict()

key='l1'
widgetList[key]=widgets.FloatSlider(description=key,
            min=0.0,max=1.0,value=Data[key],step=0.01,continuous_update=False, orientation='horizontal',
            readout_format='0.3f')
key='r1'
widgetList[key]=widgets.FloatSlider(description=key,
            min=0.0,max=1.0,value=Data[key],step=0.01,continuous_update=False, orientation='horizontal',
            readout_format='0.3f')
key='l2'
widgetList[key]=widgets.FloatSlider(description=key,
            min=0.0,max=1.0,value=Data[key],step=0.01,continuous_update=False, orientation='horizontal',   
           readout_format='0.3f')
key='r2'
widgetList[key]=widgets.FloatSlider(description=key,
            min=0.0,max=1.0,value=Data[key],step=0.01,continuous_update=False, orientation='horizontal',
            readout_format='0.3f')
key='strands'
widgetList[key]=widgets.IntSlider(description=key,
            min=1,max=5,value=Data[key],step=1,continuous_update=False, orientation='horizontal',
            readout_format='d')


for w in widgetList.values():
    w.observe(handle_change,'value')
    if w.orientation=='horizontal':
      w.layout.width='5in'
    else:
      w.layout.height='4in'

Layout=VBox(
        [
        HBox([VBox([widgetList['l1'], widgetList['r1'],widgetList['l2'],widgetList['r2'],widgetList['strands']],), output1,],),
        output2,
        output3
        ])
update_plot();

## Interactive Section

In [3]:
display(Layout);